In [1]:
import pickle
%load_ext autoreload
%autoreload 2
import scalable_gps
from scalable_gps.kernels import Matern32Kernel

import jax.numpy as jnp
import gpytorch as gpt
import numpy as np
import torch
import gpflux
# filepath = '/home/shreyaspadhy_gmail_com/scalable-gaussian-processes/scalable_gps/results/exact_gp_results/dataset_pol.evaltol_0.001.kernel_matern.loverank_200.lr_0.1.model_exact.numiter_100.seed_0.traintol_1.0.pkl'


# with open(filepath, 'rb') as f:
#     data = pickle.load(f)

# print(data)

/home/shreyaspadhy_gmail_com/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-06 16:01:42.324373: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-04-06 16:01:42.356741: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-06 16:01:43.101192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-

In [2]:
import jax.random as jr
import uci_datasets
import gpytorch

from data import get_dataset

train_ds, test_ds = get_dataset('yacht', normalise=True)

print(train_ds.x.shape)


# model.likelihood.noise_covar._set_noise(0.0007456933963112533)
# model.covar_module._set_outputscale(0.07279852032661438)
# model.covar_module.base_kernel._set_lengthscale(torch.from_numpy(np.asarray([0.40078604, 0.47861066, 0.99160534, 1.8033762,  1.3364722,  3.6816106,
#   3.6552188,  6.462375,   6.962077,   4.8550777,  2.6934228,  2.2448704,
#   5.32376,    5.58768,    6.0296025,  3.2405155,  6.1585517,  6.8495193,
#   5.000094,   5.8355484,  5.228479,   6.537194,   6.9643164,  5.146025,
#   5.5808063,  5.1676707])))""


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        kernel = gpt.kernels.MaternKernel(nu=1.5, ard_num_dims=6)
        # lengthscales = torch.from_numpy(np.asarray([0.17658784985542297, 0.19303946197032928, 0.30149734020233154, 0.3629351556301117, 0.2601526975631714, 1.06342351436615, 1.1373639106750488, 3.021310567855835, 3.348904848098755, 1.3357858657836914, 1.2393946647644043, 1.1090068817138672, 2.8051507472991943, 2.5480997562408447, 2.2725119590759277, 0.9565914869308472, 2.9568326473236084, 3.3208203315734863, 3.3087961673736572, 2.968641996383667, 2.4997735023498535, 3.0178418159484863, 3.307901620864868, 1.6029528379440308, 3.3901262283325195, 3.302780866622925]))
        
#         lengthscales = torch.from_numpy(np.asarray([0.40078604, 0.47861066, 0.99160534, 1.8033762,  1.3364722,  3.6816106,
#   3.6552188,  6.462375,   6.962077,   4.8550777,  2.6934228,  2.2448704,
#   5.32376,    5.58768,    6.0296025,  3.2405155,  6.1585517,  6.8495193,
#   5.000094,   5.8355484,  5.228479,   6.537194,   6.9643164,  5.146025,
#   5.5808063,  5.1676707]))
        
        # ('os', '0.09691217'), ('noise', '0.00168458'), ('ls', '1.88330865')
        # kernel.lengthscale = 1.88330865
        # kernel.lengthscale = lengthscales
        # print(f'lengtscales: {kernel.lengthscale}')
        
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)
        # self.covar_module.outputscale(1.)
        # print(f'output_scale : ', self.covar_module.outputscale)
        # self.covar_module.outputscale = 0.07279852032661438
        # self.covar_module.outputscale = 0.10929246991872787
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


x = torch.from_numpy(np.array(train_ds.x)).float()
y = torch.from_numpy(np.array(train_ds.y)).float()

test_x = torch.from_numpy(np.array(test_ds.x)).float()
test_y = torch.from_numpy(np.array(test_ds.y)).float()


# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# log(1 + e(x)) = lg(e^10 - 1)
# likelihood.noise_covar.noise = 0.0016845794161781669
# likelihood.noise_covar.noise =0.00807290431112051
# likelihood.noise_covar.noise = 0.0007456933963112533
print('noise : ', likelihood.noise_covar.noise)

model = ExactGPModel(x, y, likelihood)

print(model)

# Overwrite quantities
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    latent_pred = model(test_x)
    
print(latent_pred)

test_rmse = torch.sqrt(torch.mean(torch.pow(latent_pred.mean - test_y, 2)))

from eval_utils import RMSE

test_y_jax = jnp.array(test_y.numpy())
mean_y_jax = jnp.array(latent_pred.mean.numpy())
print(RMSE(test_y_jax, mean_y_jax, train_ds.mu_y, train_ds.sigma_y))

print(test_rmse)

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 500
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(x)
    # Calc loss and backprop gradients
    loss = -mll(output, y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   noise: %.3f output_scale: %.3f' % (
        i + 1, training_iter, loss.item(),
        # model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item(),    \
        model.covar_module.outputscale.item()
    ))
    optimizer.step()


# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    latent_pred = model(test_x)
    
print(latent_pred)


test_rmse = torch.sqrt(torch.mean(torch.pow(latent_pred.mean - test_y, 2)))

from eval_utils import RMSE

test_y_jax = jnp.array(test_y.numpy())
mean_y_jax = jnp.array(latent_pred.mean.numpy())
print(RMSE(test_y_jax, mean_y_jax, train_ds.mu_y, train_ds.sigma_y))

print(test_rmse)
# # kernel_gpt = gpt.kernels.MaternKernel(nu=1.5)
# kernel_gpt = gpt.kernels.ScaleKernel(gpt.kernels.MaternKernel(nu=1.5))



# # data = jr.normal(key=jr.PRNGKey(0), shape=(10, 1))

yacht dataset, N=308, d=6
(278, 6)
noise :  tensor([0.6932], grad_fn=<AddBackward0>)
ExactGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): MaternKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)
MultivariateNormal(loc: torch.Size([30]))


/home/shreyaspadhy_gmail_com/.local/lib/python3.8/site-packages/jax/_src/api_util.py:183: SyntaxWarning: Jitted function has static_argnums=(3, 4), but only accepts 4 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


1.1297213
tensor(0.6141)
Iter 1/500 - Loss: 1.137   noise: 0.693 output_scale: 0.693
Iter 2/500 - Loss: 1.098   noise: 0.644 output_scale: 0.644
Iter 3/500 - Loss: 1.059   noise: 0.598 output_scale: 0.599
Iter 4/500 - Loss: 1.019   noise: 0.554 output_scale: 0.557
Iter 5/500 - Loss: 0.979   noise: 0.513 output_scale: 0.521
Iter 6/500 - Loss: 0.939   noise: 0.474 output_scale: 0.491
Iter 7/500 - Loss: 0.898   noise: 0.437 output_scale: 0.468
Iter 8/500 - Loss: 0.856   noise: 0.402 output_scale: 0.452
Iter 9/500 - Loss: 0.814   noise: 0.370 output_scale: 0.443
Iter 10/500 - Loss: 0.772   noise: 0.339 output_scale: 0.439
Iter 11/500 - Loss: 0.729   noise: 0.311 output_scale: 0.439
Iter 12/500 - Loss: 0.687   noise: 0.285 output_scale: 0.441
Iter 13/500 - Loss: 0.644   noise: 0.260 output_scale: 0.444
Iter 14/500 - Loss: 0.601   noise: 0.238 output_scale: 0.447
Iter 15/500 - Loss: 0.558   noise: 0.217 output_scale: 0.448
Iter 16/500 - Loss: 0.516   noise: 0.197 output_scale: 0.447
Iter 17/

In [3]:
print(model.covar_module.base_kernel.lengthscale)
print(np.sqrt(model.likelihood.noise.item()))
print(np.sqrt(model.covar_module.outputscale.item()))

tensor([[15.1872,  9.7537, 19.5300, 13.5691, 16.2901,  5.7411]],
       grad_fn=<SoftplusBackward0>)
0.01227042308395588
2.4222777985351653


In [79]:



# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

print(model.covar_module.base_kernel.lengthscale.detach().numpy())
print(model.likelihood.noise.item())
print(likelihood.noise.item())
print(model.covar_module.outputscale.item())


print(model.covar_module.base_kernel.lengthscale.detach().numpy())
print(model.likelihood.noise.item())
print(likelihood.noise.item())
print(model.covar_module.outputscale.item())


# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    latent_pred = model(test_x)
    
print(latent_pred)


test_rmse = torch.sqrt(torch.mean(torch.pow(latent_pred.mean - test_y, 2)))

from eval_utils import RMSE

test_y_jax = jnp.array(test_y.numpy())
mean_y_jax = jnp.array(latent_pred.mean.numpy())
print(RMSE(test_y_jax, mean_y_jax, train_ds.mu_y, train_ds.sigma_y))

print(test_rmse)

# insert commas into followign list - [0.40078604 0.47861066 0.99160534 1.8033762  1.3364722  3.6816106
#   3.6552188  6.462375   6.962077   4.8550777  2.6934228  2.2448704
#   5.32376    5.58768    6.0296025  3.2405155  6.1585517  6.8495193
#   5.000094   5.8355484  5.228479   6.537194   6.9643164  5.146025
#   5.5808063  5.1676707 ]
  
# model.covar_module.base_kernel.lengthscale = [0.40078604, 0.47861066, 0.99160534, 1.8033762,  1.3364722,  3.6816106,
#   3.6552188,  6.462375,   6.962077,   4.8550777,  2.6934228,  2.2448704,
#   5.32376,    5.58768,    6.0296025,  3.2405155,  6.1585517,  6.8495193,
#   5.000094,   5.8355484,  5.228479,   6.537194,   6.9643164,  5.146025,
#   5.5808063,  5.1676707]

[[0.40078604 0.47861066 0.99160534 1.8033762  1.3364722  3.6816106
  3.6552188  6.4623747  6.9620767  4.8550777  2.6934228  2.2448704
  5.32376    5.58768    6.0296025  3.2405155  6.1585517  6.8495193
  5.000094   5.8355484  5.2284794  6.537194   6.9643164  5.1460247
  5.5808063  5.1676707 ]]
0.0007456933963112533
0.0007456933963112533
0.07279852032661438
[[0.40078604 0.47861066 0.99160534 1.8033762  1.3364722  3.6816106
  3.6552188  6.4623747  6.9620767  4.8550777  2.6934228  2.2448704
  5.32376    5.58768    6.0296025  3.2405155  6.1585517  6.8495193
  5.000094   5.8355484  5.2284794  6.537194   6.9643164  5.1460247
  5.5808063  5.1676707 ]]
0.0007456933963112533
0.0007456933963112533
0.07279852032661438
MultivariateNormal(loc: torch.Size([1500]))
33.904427
tensor(0.8120)


In [ ]:
l1 = [[0.17658784985542297, 0.19303946197032928, 0.30149734020233154, 0.3629351556301117, 0.2601526975631714, 1.06342351436615, 1.1373639106750488, 3.021310567855835, 3.348904848098755, 1.3357858657836914, 1.2393946647644043, 1.1090068817138672, 2.8051507472991943, 2.5480997562408447, 2.2725119590759277, 0.9565914869308472, 2.9568326473236084, 3.3208203315734863, 3.3087961673736572, 2.968641996383667, 2.4997735023498535, 3.0178418159484863, 3.307901620864868, 1.6029528379440308, 3.3901262283325195, 3.302780866622925]]

l2 = [[0.18632957339286804, 0.18670417368412018, 0.3110752999782562, 0.38153818249702454, 0.2578975260257721, 1.0106427669525146, 1.149392008781433, 3.015958547592163, 3.4424891471862793, 1.3288651704788208, 1.2516462802886963, 0.9662008285522461, 2.8585398197174072, 2.4922971725463867, 2.5090160369873047, 0.8172422051429749, 2.799964666366577, 3.24336838722229, 3.2503981590270996, 3.061379909515381, 2.3566246032714844, 2.9445431232452393, 3.2977046966552734, 1.5283869504928589, 3.285978317260742, 3.14375638961792]]

l3 = [[0.18384207785129547, 0.18583382666110992, 0.2833012640476227, 0.3608447313308716, 0.24685616791248322, 0.9405211210250854, 1.130507230758667, 2.9164040088653564, 3.31789493560791, 1.364676594734192, 1.2581337690353394, 1.1084792613983154, 2.7693064212799072, 2.4154539108276367, 2.3878650665283203, 0.786465048789978, 2.850242853164673, 3.3356235027313232, 3.2113757133483887, 3.167024850845337, 2.1103827953338623, 3.0827345848083496, 3.291494846343994, 1.6791802644729614, 3.3651740550994873, 3.087095022201538]]